In [2]:
import os
os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Versions/4.3-arm64/Resources'
os.environ['R_LIBS_USER'] = '/Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/library'  # Replace with your path


In [3]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import random

import plotly.graph_objects as go


# Import necessary libraries
from rpy2.robjects import pandas2ri, Matrix
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [4]:
import rpy2.robjects.packages as rpackages

if not rpackages.isinstalled('limma'):
    utils = rpackages.importr('utils')
    utils.chooseCRANmirror(ind=1)  # Select the first CRAN mirror
    utils.install_packages('limma')



In [5]:
control_group = pd.read_excel('_a7ebd2b56abe41d344c62ff51f62afa6_Transciptomic-Data--Control-Group.xlsx')
control_group.head()

,Gene Name,MDAMB453 Cell Line,MDAMB231 Cell Line,MDAMB468 Cell Line,BT549 Cell Line,CAL851 Cell Line
0,1/2-SBSRNA4,3.999746,6.281882,4.706822,4.241822,5.193640
1,A1BG,7.717929,5.286117,4.167404,5.348666,5.304134
2,A1BG-AS1,5.844432,4.591649,4.500660,3.644126,5.666137
3,A1CF,5.572347,5.154188,4.330445,3.709960,5.890612
4,A2LD1,6.025314,4.483234,6.581222,6.828709,8.031257


In [6]:
drug_group = pd.read_excel('_a81fad69520a7b5be0fb70d9c6fbf6a7_Transciptomic-Data--Drug-Group.xlsx')
drug_group.head()

,Gene Name,MDAMB453 Cell Line,MDAMB231 Cell Line,MDAMB468 Cell Line,BT549 Cell Line,CAL851 Cell Line
0,1/2-SBSRNA4,4.000746,6.251882,4.720942,4.220613,5.201640
1,A1BG,7.725929,5.256117,4.179906,5.321923,5.312134
2,A1BG-AS1,5.852432,4.561649,4.514162,3.625905,5.674137
3,A1CF,5.580347,5.124188,4.343436,3.691410,5.898612
4,A2LD1,6.033314,4.453234,6.600966,6.794565,8.039257


In [7]:
def normalise_data(df):
    # compute min max normalisation for each cell line
    df = df.set_index('Gene Name')
    #normalized_df = (df - df.min()) / (df.max() - df.min())
    normalized_df = (df - df.mean()) / df.std()
    
    return normalized_df.reset_index()

def log_transform_data(df):
    
    df = df.set_index('Gene Name')
    log_df = np.log2(df + 1)

    return log_df.reset_index()


def preprocess_data(df):
    
    df = normalise_data(df)
    #df = log_transform_data(df)
    df = df.set_index('Gene Name')     
    # Remove lowly expressed genes (genes with expression values below the 25th percentile)
    #df = df[df.apply(lambda x: x > x.quantile(0.25)).all(axis=1)]

    # Remove genes with zero variance
    df = df.loc[df.var(axis=1) != 0]
    return df.reset_index()



def differential_expression(control_df, drug_df):
    de_results = []
    control_df.set_index('Gene Name',inplace = True)
    drug_df.set_index('Gene Name',inplace = True)
    fold_change_df  = compute_fold_change(control_df.reset_index(), drug_df.reset_index())
    for gene in control_df.index:
        # Get expression values for this gene in all cell lines
        control_group = control_df.loc[gene]
        drug_group = drug_df.loc[gene]

        # Perform t-test
        t_stat, p_value = ttest_ind(control_group, drug_group,equal_var=False, nan_policy='raise')

        # Calculate fold change
        fold_change = fold_change_df.loc[gene].mean()

        # Create a DataFrame with results
        results = pd.DataFrame({
            'Gene': gene,
            't_stat': t_stat,
            'p_value': p_value,
            'fold_change': fold_change
        }, index=[0])

        de_results.append(results)

    # Combine all results into a single DataFrame
    de_results = pd.concat(de_results)

    # Adjust p-values using Benjamini-Hochberg procedure (the qvalues), accounting for multiple testing
    de_results['adjusted_p_value'] = multipletests(de_results['p_value'], method='fdr_bh')[1]

    return de_results


def compute_fold_change(control_df, drug_df):

    control_df.set_index('Gene Name',inplace = True)
    drug_df.set_index('Gene Name',inplace = True)

    fold_change_df = pd.DataFrame(index=control_df.index)

    cell_lines = ['MDAMB453', 'MDAMB231', 'MDAMB468', 'BT549', 'CAL851']

    for cell_line in cell_lines:
        control_col = f'{cell_line} Cell Line'
        drug_col = f'{cell_line} Cell Line'
        
        # Calculate fold change (drug/control) for each gene
        fold_change = drug_df[drug_col] / control_df[control_col] -1 
        
        # Add fold change values to the dataframe
        fold_change_df[cell_line + ' Fold Change'] = fold_change

    # Replace infinities or -infinities with NaN
    fold_change_df.replace([np.inf, -np.inf], 1, inplace=True)

    return fold_change_df

In [8]:
# Preprocess the data
control_group_norm = preprocess_data(control_group)
drug_group_norm = preprocess_data(drug_group)

In [9]:
control_group_norm = control_group_norm.set_index('Gene Name')
drug_group_norm = drug_group_norm.set_index('Gene Name')

In [10]:

# Import R packages
limma = importr('limma')

# Activate pandas conversion for rpy2
pandas2ri.activate()

# Merge the control and drug dataframes
control_group_norm.columns = control_group_norm.columns + "-control"
drug_group_norm.columns = drug_group_norm.columns + "-drug"

data_df = pd.concat([control_group_norm, drug_group_norm], axis=1, )

# Create a binary design matrix
design_df = pd.DataFrame({'control': [1]*control_group_norm.shape[1] + [0]*drug_group_norm.shape[1],
                          'drug': [0]*control_group_norm.shape[1] + [1]*drug_group_norm.shape[1]})
assert design_df.shape[0] == data_df.shape[1], "Design matrix must have the same number of rows as the data matrix has columns"

print(data_df.shape)
print(design_df.shape)
# Convert pandas dataframes to rpy2 Matrix objects
data_r = pandas2ri.py2rpy(data_df)
design_r = pandas2ri.py2rpy(design_df)

# Apply lmFit
fit = limma.lmFit(data_r, design_r)

(20042, 10)
(10, 2)


In [11]:
fit = limma.eBayes(fit)

# Get differential expression results
diff_expr = limma.topTable(fit, coef = 2, number = float('inf'))

# Convert back to pandas DataFrame
de_results = robjects.conversion.rpy2py(diff_expr)


In [12]:
de_results

,logFC,AveExpr,t,P.Value,adj.P.Val,B
RPL7L1,2.513837,2.513936,34.995275,5.856260e-13,1.173712e-08,19.951738
RPL7A,3.172052,3.172317,28.403952,6.161005e-12,2.469450e-08,17.873516
C20orf24,2.649558,2.649688,27.541467,8.714690e-12,2.469450e-08,17.554546
GAPDH,3.229179,3.229458,26.894615,1.138261e-11,2.469450e-08,17.306904
MTCH1,2.699948,2.700098,26.852124,1.158659e-11,2.469450e-08,17.290375
...,...,...,...,...,...,...
GRAMD1B,0.000264,-0.000220,0.001948,9.984795e-01,9.986788e-01,-7.572035
PGGT1B,0.000328,0.000129,0.001677,9.986912e-01,9.988407e-01,-7.572035
PRRT3,0.000316,0.000145,0.001348,9.989480e-01,9.990477e-01,-7.572036
ENAH,-0.000181,-0.000420,-0.000809,9.993689e-01,9.994187e-01,-7.572036


In [49]:
# Calculate log2 fold change and -log10 adjusted p-value
de_results['log2fc'] = (de_results['logFC'])
de_results['-log10p'] = -np.log10(de_results['adj.P.Val'])

# Define significance thresholds
threshold_log2fc_neg = -1.3  # Equivalent to a fold change of 2
threshold_log2fc_pos = 2
threshold_log10p = -np.log10(0.05)  # Equivalent to an adjusted p-value of 0.05

# Create a volcano plot
fig = go.Figure()

# Add all points
fig.add_trace(go.Scatter(
    x=de_results['log2fc'],
    y=de_results['-log10p'],
    mode='markers',
    text=de_results.index,  # Display gene names on hover
    marker=dict(
        color='gray',  # Color points gray by default
        line=dict(width=1)  # Outline markers
    ),
    hovertemplate="<b>%{text}</b><br><br>log2fc: %{x}<br>-log10p: %{y}<extra></extra>",  # Customize hover text
    name=''
))

# Highlight significant points
fig.add_trace(go.Scatter(
    x=de_results.loc[((de_results['log2fc'] < threshold_log2fc_neg) |
                     (de_results['log2fc'] > threshold_log2fc_pos) )&
                     (de_results['-log10p'] > threshold_log10p), 'log2fc'],
    y=de_results.loc[((de_results['log2fc'] < threshold_log2fc_neg) |
                     (de_results['log2fc'] > threshold_log2fc_pos) )& (de_results['-log10p'] > threshold_log10p), '-log10p'],
    mode='markers',
    text=de_results.loc[((de_results['log2fc'] < threshold_log2fc_neg) |
                     (de_results['log2fc'] > threshold_log2fc_pos) )& (de_results['-log10p'] > threshold_log10p)].index,
    marker=dict(
        color='red',  # Color significant points red
        line=dict(width=1)
    ),
    hovertemplate="<b>%{text}</b><br><br>log2fc: %{x}<br>-log10p: %{y}<extra></extra>",
    name='Significant'
))

# Add lines to denote significance thresholds
fig.add_shape(type="line", x0=threshold_log2fc_pos, x1=threshold_log2fc_pos, y0=0, y1=de_results['-log10p'].max(), line=dict(color="black", width=1))
fig.add_shape(type="line", x0=threshold_log2fc_neg, x1=threshold_log2fc_neg, y0=0, y1=de_results['-log10p'].max(), line=dict(color="black", width=1))
fig.add_shape(type="line", x0=de_results['log2fc'].min(), x1=de_results['log2fc'].max(), y0=threshold_log10p, y1=threshold_log10p, line=dict(color="black", width=1))

# Update layout
fig.update_layout(
    title='Volcano plot',
    xaxis_title='log2 fold change',
    yaxis_title='-log10 adjusted p-value',
    showlegend=True,
    autosize=False,
    width=800,
    height=600,
)

# Show plot
fig.show()

In [50]:
significant_genes = list(de_results.loc[((de_results['log2fc'] < threshold_log2fc_neg) |
                        (de_results['log2fc'] > threshold_log2fc_pos) )&
                        (de_results['-log10p'] > threshold_log10p)].index)
significant_genes

['RPL7L1',
 'RPL7A',
 'C20orf24',
 'GAPDH',
 'MTCH1',
 'C6orf48',
 'USMG5',
 'CUTA',
 'NDUFB2',
 'UBA52',
 'CCDC72',
 'TMSB10',
 'RPSA',
 'MEA1',
 'DNAJC9',
 'C14orf166',
 'MYL6',
 'RPS12',
 'MT2A',
 'MRFAP1',
 'ENY2',
 'TRMT5',
 'RPL36',
 'TMEM126A',
 'CCNB1',
 'NDUFA12',
 'RPS26',
 'MALSU1',
 'DYNC1I2',
 'POMP',
 'NACAP1',
 'MRPL14',
 'CCDC56',
 'TMEM14B',
 'ATP5F1',
 'RPL26',
 'TOMM5',
 'ATP5E',
 'FBL',
 'C11orf10',
 'TPT1',
 'RACGAP1',
 'PPP1R14B',
 'TRAPPC4',
 'MYL12B',
 'PRELID1',
 'VDAC2',
 'UBE2E1',
 'MT1P2',
 'SUB1',
 'POLR2J',
 'NUCKS1',
 'C19orf53',
 'ZNFX1-AS1',
 'MTCH2',
 'YWHAG',
 'SLIRP',
 'TXNDC17',
 'RPL35A',
 'FTH1P5',
 'SPCS1',
 'TIMM8B',
 'PDZD11',
 'NDUFC2',
 'MINOS1',
 'CISD1',
 'MRPS24',
 'NDUFA4',
 'VDAC1',
 'PSMD14',
 'LAMTOR1',
 'MRPL51',
 'HIGD1A',
 'DCUN1D5',
 'MRPL36',
 'HSPA5',
 'CIRH1A',
 'RND3',
 'MORF4L1',
 'MAT2B',
 'SNHG6',
 'EIF3L',
 'PPA1',
 'ZNF706',
 'TMEM14C',
 'NIPA2',
 'MRPL32',
 'UXT',
 'PPIL1',
 'KCMF1',
 'DNAJC15',
 'KRTCAP2',
 'ESR2',
 'NDU

## Gene Set enrichment


In [51]:
#import the gene set from json file
import json
with open('KEGG_APOPTOSIS.v2023.1.Hs.json') as json_file:
    gene_set_file = json.load(json_file)

gene_set = gene_set_file['KEGG_APOPTOSIS']['geneSymbols']

In [52]:
import gseapy as gp


In [53]:
# run enrichr
# if you are only intrested in dataframe that enrichr returned, please set outdir=None
enr = gp.enrichr(gene_list= significant_genes, # or "./tests/data/gene_list.txt",
                 gene_sets=['KEGG_2021_Human'],
                 organism='human', # don't forget to set organism to the one you desired! e.g. Yeast
                 outdir=None, # don't write to disk
                )

In [54]:
enr.results.head(20)

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,KEGG_2021_Human,Ribosome,78/158,1.866747e-76,4.461526e-74,0,0,36.300434,6329.788980,RPL4;RPL5;RPL30;RPL3;RPL32;RPL34;RPLP1;RPLP0;M...
1,KEGG_2021_Human,Coronavirus disease,70/232,1.127383e-50,1.347223e-48,0,0,15.776115,1814.400765,RPL4;RPL5;RPL30;RPL3;RPL32;RPL34;RPLP1;RPLP0;R...
2,KEGG_2021_Human,Parkinson disease,70/249,2.458518e-48,1.958619e-46,0,0,14.265209,1563.816819,COX7B;NDUFA13;NDUFA12;COX4I1;PARK7;COX6A1;COX7...
3,KEGG_2021_Human,Huntington disease,72/306,7.207511e-44,4.306488e-42,0,0,11.234579,1116.027555,COX7B;NDUFA13;NDUFA12;COX4I1;CLTC;COX6A1;COX7C...
4,KEGG_2021_Human,Prion disease,65/273,5.703008e-40,2.726038e-38,0,0,11.275258,1018.859528,COX7B;NDUFA13;NDUFA12;COX4I1;COX6A1;COX7C;TUBA...
5,KEGG_2021_Human,Amyotrophic lateral sclerosis,73/364,2.145773e-39,8.547330e-38,0,0,9.149655,814.660752,COX7B;NDUFA13;NDUFA12;COX4I1;COX6A1;COX7C;ACTB...
6,KEGG_2021_Human,Pathways of neurodegeneration,77/475,9.247319e-35,3.157299e-33,0,0,7.070859,554.115967,COX7B;NDUFA13;NDUFA12;COX4I1;PARK7;COX6A1;COX7...
7,KEGG_2021_Human,Alzheimer disease,66/369,1.769485e-32,5.286337e-31,0,0,7.835021,572.834308,COX7B;NDUFA13;NDUFA12;COX4I1;COX6A1;COX7C;TUBA...
8,KEGG_2021_Human,Oxidative phosphorylation,40/133,3.269351e-29,8.681943e-28,0,0,14.910909,978.012134,NDUFB9;NDUFA13;COX7B;NDUFB8;NDUFB10;UQCRB;NDUF...
9,KEGG_2021_Human,Diabetic cardiomyopathy,43/203,1.623841e-24,3.880980e-23,0,0,9.334939,511.342271,NDUFB9;COX7B;NDUFA13;NDUFB8;NDUFB10;UQCRB;NDUF...


In [57]:
pd.DataFrame(significant_genes).to_clipboard(index=False,header=False)